# Master Fact Table (Orders Master)

This notebook builds a master fact table by combining cleaned datasets.

Goal:
- Create one analytical table that supports answering:
  1) Why customers give low ratings (dissatisfaction drivers)
  2) Where profit/revenue comes from and which categories matter most

Inputs:
- `orders_clean.csv`
- `order_items_clean.csv`
- `olist_order_payments_dataset.csv`
- `olist_order_reviews_dataset.csv`
- `olist_products_dataset.csv`
- (optional) category translation table if needed later

## Step 1: Load Clean and Raw Tables

Load the cleaned tables first, then load supporting raw tables
needed for payments, reviews, and product categories.

In [33]:
import pandas as pd
import numpy as np

orders_clean = pd.read_csv("../data/processed/orders_clean.csv")
order_items_clean = pd.read_csv("../data/processed/order_items_clean.csv")

payments = pd.read_csv("../data/raw/olist_order_payments_dataset.csv")
reviews = pd.read_csv("../data/raw/olist_order_reviews_dataset.csv")
products = pd.read_csv("../data/raw/olist_products_dataset.csv")

orders_clean.shape, order_items_clean.shape

((96478, 10), (110197, 8))

## Step 2: Aggregate Payments per Order

An order can have multiple payment records.
We aggregate them to one value per order (`order_revenue`).

In [34]:
payments_agg = payments.groupby("order_id", as_index=False)["payment_value"].sum()
payments_agg.rename(columns={"payment_value": "order_revenue"}, inplace=True)

payments_agg.head()

,order_id,order_revenue
0,00010242fe8c5a6d1ba2dd792cb16214,72.19
1,00018f77f2f0320c557190d7a144bdd3,259.83
2,000229ec398224ef6ca0657da4fc703e,216.87
3,00024acbcdf0a6daa1e931b038114c75,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04


## Step 3: Prepare Reviews per Order

We keep the review score per order to analyze dissatisfaction drivers.
If multiple reviews exist (rare), we keep the first non-null score.

In [35]:
reviews_clean = reviews[["order_id", "review_score"]].copy()

reviews_clean = (
    reviews_clean
    .dropna(subset=["review_score"])
    .drop_duplicates(subset=["order_id"])
)

reviews_clean.head()

,order_id,review_score
0,73fc7af87114b39712e6da79b0a377eb,4
1,a548910a1c6147796b98fdf73dbeba33,5
2,f9e4b658b201a9f2ecdecbb34bed034b,5
3,658677c97b385a9be170737859d3511b,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,5


## Step 4: Attach Product Category to Each Item

Join `order_items_clean` with `products` to bring `product_category_name`
into the item-level dataset.

In [36]:
items_with_cat = order_items_clean.merge(
    products[["product_id", "product_category_name"]],
    on="product_id",
    how="left"
)
# Keep purchase timestamp from orders_clean only (avoid _x/_y after merge)
items_with_cat = items_with_cat.drop(columns=["order_purchase_timestamp"], errors="ignore")

items_with_cat[["order_id", "product_id", "product_category_name"]].head()

,order_id,product_id,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,pet_shop
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,moveis_decoracao
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,perfumaria
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,ferramentas_jardim


## Step 5: Build the Master Table (Item Level)

We build the master fact table at the item level, because:
- Revenue and categories are item-based
- We can still analyze satisfaction by joining review score per order

We combine:
- orders_clean (timestamps + delivery performance)
- payments_agg (order revenue)
- reviews_clean (review score)
- items_with_cat (items + category)

In [ ]:
items_with_cat = items_with_cat.drop(columns=["order_purchase_timestamp"], errors="ignore")

fact_orders_master = (
    items_with_cat
    .merge(orders_clean, on="order_id", how="inner")
    .merge(payments_agg, on="order_id", how="left")
    .merge(reviews_clean, on="order_id", how="left")
)

fact_orders_master.shape

(110197, 19)

## Step 6: Create Key Analytical Flags

Create helper columns to support the two main questions:
1) Dissatisfaction: low rating and delay indicators
2) Revenue focus: item total value

In [38]:
fact_orders_master["is_low_rating"] = fact_orders_master["review_score"].isin([1, 2])

fact_orders_master["item_total_value"] = (
    fact_orders_master["price"] + fact_orders_master["freight_value"]
)

fact_orders_master[["review_score", "is_low_rating", "item_total_value"]].head()

,review_score,is_low_rating,item_total_value
0,5.0,False,72.19
1,4.0,False,259.83
2,5.0,False,216.87
3,4.0,False,25.78
4,5.0,False,218.04


## Step 7: Save Master Fact Table

Save the master fact table as the single source of truth
for all downstream analysis and SQL validation.

In [39]:
fact_orders_master.to_csv(
    "../data/processed/fact_orders_master.csv",
    index=False
)

fact_orders_master.shape

(110197, 21)

## Step 8: Master Table Sanity Checks

Perform basic validation checks to ensure data integrity.

In [40]:
fact_orders_master.isnull().mean().sort_values(ascending=False).head(10)

product_category_name            0.013948
review_score                     0.007505
order_approved_at                0.000136
order_delivered_customer_date    0.000073
delivery_time_days               0.000073
order_revenue                    0.000027
order_delivered_carrier_date     0.000018
shipping_limit_date              0.000000
order_item_id                    0.000000
order_id                         0.000000
dtype: float64

## Sanity Check Summary

The master fact table shows a very low level of missing values.

- Missing product categories (~1.4%) are expected due to incomplete product classification.
- Missing review scores (~0.75%) reflect delivered orders without customer feedback.
- All revenue-critical fields (price, product_id, shipping_limit_date) have **0% missing values**.

Overall, the dataset is analytically sound and suitable for
revenue analysis, customer satisfaction analysis, and SQL validation.